In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 485 (delta 66), reused 49 (delta 44), pack-reused 363
Receiving objects: 100% (485/485), 8.89 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [2]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/penguin_y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)


In [3]:
# 1. 전처리 ( 결측치 제거)
s=pd.concat([X_train,y_train],axis=1)
s=s.dropna().reset_index(drop=True)
#print(s.columns)
# 2. 사용할 데이터 선택
col_del=[]
col_num=['bill_length_mm', 'bill_depth_mm','flipper_length_mm']
col_cat=['species', 'island', 'sex']
col_y=['body_mass_g']

In [ ]:
# 원핫인코더
from sklearn.preprocessing import OneHotEncoder

ohe=OneHotEncoder()
ohe.fit(s[col_cat])

x_train_res=ohe.transform(s[col_cat])
x_test_res=ohe.transform(X_test[col_cat])

x_train_ohe=pd.DataFrame(x_train_res.todense(),columns=ohe.get_feature_names())
x_test_ohe=pd.DataFrame(x_test_res.todense(),columns=ohe.get_feature_names())
x_train_fin=pd.concat([s[col_num],x_train_ohe],axis=1)
x_test_fin=pd.concat([X_test[col_num],x_test_ohe],axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
#라벨인코더
from sklearn.preprocessing import LabelEncoder

for col in col_cat :
  lb=LabelEncoder()
  lb.fit(s[col])
  s[col]=lb.transform(s[col])
  
  lb.fit(X_test[col])
  X_test[col]=lb.transform(X_test[col])


In [17]:
# 3. 데이터 분할
from sklearn.model_selection import train_test_split
#원핫인코더 쓴거
#x_tr,x_val,y_tr,y_val=train_test_split(x_train_fin,s[col_y],test_size=0.3)
#원핫인코더 안쓴거
#x_tr,x_val,y_tr,y_val=train_test_split(s[col_num],s[col_y],test_size=0.3)
#라벨인코더 쓴거
x_tr,x_val,y_tr,y_val=train_test_split(pd.concat([s[col_num],s[col_cat]],axis=1),s[col_y],test_size=0.3)

In [18]:
# 4. 프리프로세싱
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(x_tr[col_num])
x_tr[col_num]=scaler.transform(x_tr[col_num])
x_val[col_num]=scaler.transform(x_val[col_num])
X_test[col_num]=scaler.transform(X_test[col_num])

In [21]:
# 5. 모델링
from sklearn.ensemble import RandomForestRegressor
reg=RandomForestRegressor()
reg.fit(x_tr,y_tr)
pred=reg.predict(x_val)
# 6. 평가
from sklearn.metrics import mean_squared_error, accuracy_score
mse=mean_squared_error(y_val,pred)
print(mse)


88387.86607142857


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [22]:
# 5. 모델링
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x_tr,y_tr)
pred=reg.predict(x_val)
# 6. 평가
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_val,pred)
print(mse)

129455.11900886746


In [23]:
# 5. 모델링
from lightgbm import LGBMRegressor
reg=LGBMRegressor()
reg.fit(x_tr,y_tr)
pred=reg.predict(x_val)
# 6. 평가
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_val,pred)
print(mse)

101249.2438534572


In [24]:
# 5. 모델링
from xgboost import XGBRegressor
reg=XGBRegressor()
reg.fit(x_tr,y_tr)
pred=reg.predict(x_val)
# 6. 평가
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_val,pred)
print(mse)

[07:56:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
97158.17737120134


In [29]:
y_pred=reg.predict(pd.concat([X_test[col_num],X_test[col_cat]],axis=1))

In [ ]:
pd.DataFrame({'body_mass_g': y_pred[:,0]}).to_csv('./yemoonsaBigdata/res/004000000.csv', index=False)